# 1. Import Library

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

# 2. Read data

In [ ]:
# with open('train_vi.json', 'r') as f:
#     train_js = json.load(f)

In [ ]:
train_csv = pd.read_csv("QRdata/vi_v2_train.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'vi_v2_train.csv'

# 4. Model

In [ ]:
!pip install transformers datasets

In [ ]:
# !pip install transformers[torch] accelerate -U

In [ ]:
pip install torch

In [ ]:
!pip install wandb

In [ ]:
pip install rouge_score

In [ ]:
from transformers import AutoTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import wandb
from datasets import Dataset, load_metric

In [ ]:
wandb.login(key = "ooo")
wandb.init(project='BARTPHO')


In [ ]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-large-vietnews-summarization")
# model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-large-vietnews-summarization")
# model.cuda()

# sentence = "VietAI là tổ chức phi lợi nhuận với sứ mệnh ươm mầm tài năng về trí tuệ nhân tạo và xây dựng một cộng đồng các chuyên gia trong lĩnh vực trí tuệ nhân tạo đẳng cấp quốc tế tại Việt Nam."
# text =  "vietnews: " + sentence + " </s>"
# encoding = tokenizer(text, return_tensors="pt")
# input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
# outputs = model.generate(
#     input_ids=input_ids, attention_mask=attention_masks,
#     max_length=256,
#     early_stopping=True
# )
# for output in outputs:
#     line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
#     print(line)


In [ ]:
# sentence = "Xin chào tôi là một con thỏ. Thỏ nào đấy nhỉ?"
# text =  "vietnews: " + sentence + " </s>"
# encoding = tokenizer(text, return_tensors="pt")
# input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
# outputs = model.generate(
#     input_ids=input_ids, attention_mask=attention_masks,
#     max_length=256,
#     early_stopping=True
# )
# for output in outputs:
#     line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
#     print(line)

In [ ]:
import pandas as pd
from datasets import Dataset

# Load the dataset
file_path = './vi_v2_train.csv'
data = pd.read_csv(file_path)

# Drop any unnamed columns if present
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

# Split the dataset into train and validation
train_data = data.sample(frac=0.8, random_state=42)
val_data = data.drop(train_data.index)

# Convert the dataframe to a Dataset
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-large-vietnews-summarization")

# Tokenize the inputs and outputs
def tokenize_function(examples):
    model_inputs = tokenizer(examples['input'], max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['output'], max_length=128, truncation=True, padding="max_length")
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Tokenize the datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["input", "output"])
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=["input", "output"])


In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-large-vietnews-summarization")
# model.cuda()

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=10_000,
    eval_steps=10_000,
    logging_dir='./logs',
    logging_steps=500,
    report_to="wandb"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [ ]:
trainer.train()

In [ ]:
# Save the model
trainer.save_model("./fine-tuned-bartpho")

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)